In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'papa-skin:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4715525%2F8006502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240504%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240504T160053Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4b845e7237a41092580f041b4b724b779e7f13d4d9968b3d5956b2834bc541226d40bd946bab0833d940526029fa62167cf33a550526c82a2cfe18c0fad4d800bdf75be12a5234550352c241ec808cba83ebc511a64efd643f85ef0dd6618127727a4f1abb9a78b34d501deec32b0fcdc259d1633c144d9720cde805fb61253198a9e86f55b5760e0bc0dc3db0e016a3e580699862f3b08961afb8dc0893cbde52a9a7c1556ab5021fbda3e2c7e0c70ebfe78e41c5a2d79678315d8fb86f5fafd0dc59b8a93d1a3e04b14610135e2fb64f0de0fd079f433737c5bd69c3d40165eaf416ada7fe5e3a95965549df1e53fbdeca737c43b26b0d406cbcb98e532e3c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 2947051697 bytes downloaded
Downloaded and uncompressed: papa-skin
Data source import complete.


In [ ]:
from keras import Model
from keras.layers import Dense, Flatten, concatenate
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, ReLU, Concatenate, GlobalAveragePooling2D, Dense, Reshape, Activation
from keras.applications import DenseNet201
import tensorflow as tf
from keras import layers, Model, Input
from keras.layers import GlobalAveragePooling2D, Reshape, Conv2D, Multiply, SeparableConv2D, LayerNormalization, Dropout, BatchNormalization
from keras.layers import Lambda, DepthwiseConv2D, BatchNormalization, Dropout, Reshape, UpSampling2D, Multiply
from keras.layers import Conv2D, Multiply
from keras import backend as K
from keras.callbacks import ModelCheckpoint,EarlyStopping
import os
import tensorflow as tf
from tensorflow import keras

In [ ]:
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info

In [ ]:
class CustomDataGenerator:
    def __init__(self, directory1, directory2, batch_size=16, img_height=224, img_width=224):
        self.directory1 = directory1
        self.directory2 = directory2
        self.batch_size = batch_size
        self.img_height = img_height
        self.img_width = img_width
        self.classes = sorted(os.listdir(directory1))
        self.num_classes = len(self.classes)
        self.class_to_index = {self.classes[i]: i for i in range(self.num_classes)}
        self.index_to_class = {v: k for k, v in self.class_to_index.items()}


        self.file_paths1 = []
        self.file_paths2 = []
        self.labels = []

        for class_name in self.classes:
            class_dir1 = os.path.join(directory1, class_name)
            class_files1 = os.listdir(class_dir1)
            self.file_paths1.extend([os.path.join(class_dir1, file) for file in class_files1])
            class_dir2 = os.path.join(directory2, class_name)
            class_files2 = os.listdir(class_dir2)
            self.file_paths2.extend([os.path.join(class_dir2, file) for file in class_files2])
            self.labels.extend([self.class_to_index[class_name]] * len(class_files1))

        self.num_samples = len(self.file_paths1)

    def parse_image(self, img_path1, img_path2, label):
        img1 = tf.io.read_file(img_path1)
        img1 = tf.image.decode_jpeg(img1, channels=3)
        img1 = tf.image.resize(img1, [self.img_height, self.img_width])
        img1 = tf.keras.applications.densenet.preprocess_input(img1)  # Or any other preprocessing you need
        img2 = tf.io.read_file(img_path2)
        img2 = tf.image.decode_jpeg(img2, channels=3)
        img2 = tf.image.resize(img2, [self.img_height, self.img_width])
        img2 = tf.keras.applications.densenet.preprocess_input(img2)
        return (img1, img2), label

    def create_dataset(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.file_paths1, self.file_paths2, self.labels))
        dataset = dataset.shuffle(buffer_size=self.num_samples)
        dataset = dataset.map(self.parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.batch(self.batch_size)
        dataset = dataset.repeat()
        dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        return dataset

In [ ]:
class CustomDataGenerator_test:
    def __init__(self, directory1, directory2, batch_size=16, img_height=224, img_width=224):
        self.directory1 = directory1
        self.directory2 = directory2
        self.batch_size = batch_size
        self.img_height = img_height
        self.img_width = img_width
        self.classes = sorted(os.listdir(directory1))
        self.num_classes = len(self.classes)
        self.class_to_index = {self.classes[i]: i for i in range(self.num_classes)}
        self.index_to_class = {v: k for k, v in self.class_to_index.items()}


        self.file_paths1 = []
        self.file_paths2 = []
        self.labels = []

        for class_name in self.classes:
            class_dir1 = os.path.join(directory1, class_name)
            class_files1 = os.listdir(class_dir1)
            self.file_paths1.extend([os.path.join(class_dir1, file) for file in class_files1])
            class_dir2 = os.path.join(directory2, class_name)
            class_files2 = os.listdir(class_dir2)
            self.file_paths2.extend([os.path.join(class_dir2, file) for file in class_files2])
            self.labels.extend([self.class_to_index[class_name]] * len(class_files1))

        self.num_samples = len(self.file_paths1)

    def parse_image(self, img_path1, img_path2, label):
        img1 = tf.io.read_file(img_path1)
        img1 = tf.image.decode_jpeg(img1, channels=3)
        img1 = tf.image.resize(img1, [self.img_height, self.img_width])
        img1 = tf.keras.applications.densenet.preprocess_input(img1)  # Or any other preprocessing you need
        img2 = tf.io.read_file(img_path2)
        img2 = tf.image.decode_jpeg(img2, channels=3)
        img2 = tf.image.resize(img2, [self.img_height, self.img_width])
        img2 = tf.keras.applications.densenet.preprocess_input(img2)
        return (img1, img2), label

    def create_dataset(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.file_paths1, self.file_paths2, self.labels))
        dataset = dataset.map(self.parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.batch(self.batch_size)
        dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        return dataset

In [ ]:
train_data1 = '/kaggle/input/papa-skin/Papa_skin/HED_edge_maps'
train_data2 = '/kaggle/input/papa-skin/Papa_skin/train_dir'
batch_size = 16
img_height = 224
img_width = 224

train_data_generator = CustomDataGenerator(directory1=train_data1, directory2=train_data2, batch_size=batch_size, img_height=img_height, img_width=img_width)
train_set = train_data_generator.create_dataset()

In [ ]:
test_data1 = '/kaggle/input/papa-skin/Papa_skin/HED_test'
test_data2 = '/kaggle/input/papa-skin/Papa_skin/test_dir'
batch_size = 16
img_height = 224
img_width = 224

test_data_generator = CustomDataGenerator_test(directory1=test_data1, directory2=test_data2, batch_size=batch_size, img_height=img_height, img_width=img_width)
test_set = test_data_generator.create_dataset()

In [ ]:
def dense_block(x, n_layers, growth_rate):
    inputs = x
    for _ in range(n_layers):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(growth_rate, (3, 3), strides=1, padding='same')(x)
        inputs = Concatenate()([inputs, x])
    return inputs

In [ ]:
def transition_block(x, n_filters):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(n_filters, (1, 1), strides=1, padding='same')(x)
    x = MaxPooling2D((2, 2), strides=2)(x)
    return x

In [ ]:
input_shape = (224, 224, 3)
num_DT = 2

In [ ]:
# Branch 1

# Input layer
input1 = Input(shape=input_shape)

# Initial convolution
x = Conv2D(64, (3, 3), strides=2, padding='same')(input1)
x = BatchNormalization()(x)
x = ReLU()(x)

# Dense blocks and transition blocks
for _ in range(num_DT):
    x = dense_block(x, 4, 12)
    x = transition_block(x, 128)
edge_features_E = x

In [ ]:
def cssa(input_tensor, num_heads=5):
    # Split input into multiple heads
    input_shape = input_tensor.shape
    depth = input_shape[-1]
    # Adjust depth to ensure it is evenly divisible by num_heads
    new_depth = depth + (num_heads - (depth % num_heads))

    # Modify input tensor depth if necessary
    if new_depth != depth:
        input_tensor = tf.keras.layers.Conv2D(new_depth, kernel_size=(1, 1))(input_tensor)

    # Calculate depth per head
    depth_per_head = new_depth // num_heads


    # Channel attention
    channel_avg = layers.GlobalAveragePooling2D()(input_tensor)
    channel_max = layers.Lambda(lambda x: tf.reduce_max(x, axis=[1, 2]))(input_tensor)
    channel_concat = layers.Concatenate(axis=1)([channel_avg, channel_max])

    channel_dense1 = layers.Dense(units=depth // num_heads, activation='sigmoid')(channel_concat)
    channel_attention = layers.Reshape((1, 1, depth // num_heads))(channel_dense1)

    channel_attention = layers.UpSampling2D(size=(input_tensor.shape[1], input_tensor.shape[2]))(channel_attention)

    # Spatial attention
    spatial_avg = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_tensor)
    spatial_max = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_tensor)

    spatial_concat = layers.Concatenate(axis=-1)([spatial_avg, spatial_max])
    spatial_conv = layers.Conv2D(filters=num_heads, kernel_size=3, padding='same', activation='sigmoid')(spatial_concat)

    spatial_attention = layers.Lambda(lambda x: tf.split(x, num_heads, axis=-1))(spatial_conv)



    attention_map = layers.Multiply()([channel_attention] + spatial_attention)
    attention_map = layers.Conv2D(filters=1, kernel_size=3, padding='same', activation='sigmoid')(attention_map)


    # Apply attention to modulate the input features
    feature_map = layers.Multiply()([input_tensor, attention_map])
    feature_map = BatchNormalization()(feature_map)
    feature_map = Dropout(0.02)(feature_map)


    return feature_map, attention_map

In [ ]:
#Branch 2
dropout_rate = 0.02
l2_reg = 0.001

In [ ]:
input2 = Input(shape=input_shape)

In [ ]:
densenet = tf.keras.applications.DenseNet201(
    include_top=True,
    weights="imagenet",
    input_tensor=input2,
    input_shape=input_shape,
    pooling=None,

)
# Exclude the last 28 layers of the model.
dermo_features = densenet.layers[-28].output

82524592/82524592 [==============================] - 0s 0us/step


In [ ]:
dermo_features = BatchNormalization()(dermo_features)
dermo_features = Dropout(0.02)(dermo_features)
# attention block
feature_maps, attention_map_A = cssa(dermo_features)
feature_maps = BatchNormalization()(feature_maps)
feature_maps = Dropout(0.02)(feature_maps)

feature_maps = SeparableConv2D( 512, kernel_size=(3, 3), activation='sigmoid',
            depthwise_initializer='he_normal', pointwise_initializer='he_normal',
            depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
            pointwise_regularizer=tf.keras.regularizers.l2(l2_reg),
            padding='same')(feature_maps)
feature_maps = BatchNormalization()(feature_maps)
feature_maps = Dropout(0.02)(feature_maps)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class ResizeLayer(Layer):
    def __init__(self, size, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)
        self.size = size

    def call(self, inputs):
        return tf.image.resize(inputs, self.size)

In [ ]:
import tensorflow as tf
from tensorflow.keras.constraints import MinMaxNorm

class BetaLayer(tf.keras.layers.Layer):
    def __init__(self, initializer="custom_initializer", **kwargs):
        super(BetaLayer, self).__init__(**kwargs)
        self.beta_initializer = initializer

    def build(self, input_shape):
        self.beta = self.add_weight(
            name='beta',
            shape=(1,),
            initializer=self.beta_initializer,
            trainable=True,
            constraint=MinMaxNorm(min_value=0.0, max_value=1.0)  # Constraining beta to [0, 1]
        )

    def call(self, inputs):
        multiplicative_input, additive_input = inputs
        return self.beta * multiplicative_input + (1 - self.beta) * additive_input

    def get_config(self):
        config = super(BetaLayer, self).get_config()
        config.update({"initializer": self.beta_initializer})
        return config

# Custom initializer to initialize beta to 0.5
def custom_initializer(shape, dtype=None):
    return tf.constant(0.5, shape=shape, dtype=dtype)

In [ ]:
def enhanced_gating_mechanism(attention_maps, smoothed_edges, beta_initializer_value):
    # Generate gating signal using a convolution followed by a sigmoid activation
    gated_signal = layers.Conv2D(filters=smoothed_edges.shape[-1] // 2, kernel_size=(1, 1), padding='SAME', activation='sigmoid')(smoothed_edges)

    # Learnable parameter for blending multiplicative and additive modulation
    beta = tf.Variable(initial_value=tf.constant_initializer(value=beta_initializer_value)(shape=(1,)),
                       trainable=True, dtype=tf.float32)

    # Hybrid modulation
    expanded_attention_maps = ResizeLayer(size=(28, 28))(attention_maps)
    multiplicative_modulation = layers.Multiply()([gated_signal, expanded_attention_maps])
    additive_modulation =  expanded_attention_maps

    combined_attention = beta * multiplicative_modulation + (1 - beta) * additive_modulation

    return combined_attention

In [ ]:
enhanced_attention_maps = enhanced_gating_mechanism(attention_map_A, edge_features_E)

In [ ]:
def aggregate_features_with_attention(feature_maps, modulated_attention_maps, dermo_features):

    modulated_attention_maps = Conv2D(filters=128, kernel_size=(1, 1), padding='same')(modulated_attention_maps)
    modulated_attention_maps = (MaxPooling2D(pool_size=(2, 2),padding="same")(modulated_attention_maps))
    modulated_attention_maps = (MaxPooling2D(pool_size=(2, 2),padding="same")(modulated_attention_maps))
    guided_feature_maps = Multiply()([dermo_features, modulated_attention_maps])
    guided_feature_maps = Concatenate()([guided_feature_maps, feature_maps])
    guided_feature_maps = BatchNormalization()(guided_feature_maps)
    guided_feature_maps = Dropout(0.02)(guided_feature_maps)
    return guided_feature_maps

In [ ]:
guided_features = aggregate_features_with_attention(feature_maps, enhanced_attention_maps, dermo_features)

In [ ]:
guided_features = Activation('relu')(guided_features)
guided_features = Dropout(0.02)(guided_features)
output = Flatten()(guided_features)
output = Dense(7, activation='softmax')(output)

In [ ]:
model = Model(inputs=[input1, input2], outputs=output)

In [ ]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.001,epsilon=0.1)

model.compile(optimizer=opt1,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback

checkpoint = ModelCheckpoint(filepath = '/content/drive/MyDrive/Papa/Final/EDB_Net.weights.h5',monitor='val_accuracy',
                             save_best_only=True,save_weights_only=True)

early_stop = EarlyStopping(monitor='val_accuracy', mode='min',patience=50, min_delta=0.001)

reduce_learning_rate=ReduceLROnPlateau(monitor = "val_accuracy", patience=3, verbose=1, factor=0.5, min_lr=0.000001, cooldown=2)

In [ ]:
class BetaLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        beta_value = None
        for layer in self.model.layers:
            if isinstance(layer, BetaLayer):
                beta_value = layer.beta.numpy()
                break
        print(f"Epoch {epoch}: beta = {beta_value}")

In [ ]:
history = model.fit(train_set,
                    steps_per_epoch = total_train_samples // batch_size,
                    epochs=60,
                    verbose=1,
                    validation_data=test_set,
                    validation_steps = total_valid_samples // batch_size,
                    callbacks=[checkpoint, reduce_learning_rate, early_stop, BetaLogger()])

In [ ]:
from tensorflow.keras import models
model.load_weights('/content/drive/MyDrive/Papa/Final/EDB_Net.weights.h5')

In [ ]:
import numpy as np
# Get true labels from the test dataset
y_true = np.concatenate([label.numpy() for _, label in test_set])

In [ ]:
predictions = model.predict(test_set)

In [ ]:
y_pred = np.argmax(predictions, axis=1)

In [ ]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report:")
print(report)